In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
bettycxh06_apnea_ecg_database_path = kagglehub.dataset_download('bettycxh06/apnea-ecg-database')

print('Data source import complete.')


Data source import complete.


In [2]:
!pip install neurokit2
!pip install wfdb

In [3]:
import os
import numpy as np
import pandas as pd
import neurokit2 as nk
import wfdb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Use Random Forest Classifier

In [4]:
import os
import numpy as np
import neurokit2 as nk
import wfdb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Define the path to the dataset
base_path = '/root/.cache/kagglehub/datasets/bettycxh06/apnea-ecg-database/versions/7/apnea-ecg-database-1.0.0'
data_path = os.path.join(base_path, 'apnea-ecg-database-1.0.0')

# List of ECG record names to process
record_names = [
    "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
    "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
    "b01", "b02", "b03", "b04", "b05",
    "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
]

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Process each ECG record
for record_name in record_names:
    try:
        # Construct the full path to the record
        record_path = os.path.join(data_path, record_name)
        print(f"Loading record: {record_name}")

        # Load the ECG signal
        record = wfdb.rdrecord(record_path)
        ecg_signal = record.p_signal[:, 0]  # Assuming ECG is in the first channel

        # Load the annotations
        annotation = wfdb.rdann(record_path, 'apn')
        labels = annotation.symbol  # 'A' for Apnea, 'N' for Normal
        print(f"Loaded annotations: {labels}")

        # Clean the ECG signal
        ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=record.fs)

        # Detect R-peaks
        r_peaks = nk.ecg_findpeaks(ecg_cleaned, sampling_rate=record.fs)['ECG_R_Peaks']

        # Check if sufficient R-peaks are detected
        if len(r_peaks) < 2:
            print(f"Warning: Insufficient R-peaks detected in {record_name}. Skipping.")
            continue

        # Extract RR intervals (time between R-peaks) in milliseconds
        rr_intervals = np.diff(r_peaks) / record.fs * 1000

        # Extract R-peak amplitudes
        r_amplitudes = ecg_cleaned[r_peaks]

        # Aggregate features for each segment
        for i in range(min(len(labels), len(rr_intervals))):
            if labels[i] in ['A', 'N']:
                # Calculate mean and standard deviation of RR intervals up to the current index
                rr_mean = np.mean(rr_intervals[:i + 1])
                rr_std = np.std(rr_intervals[:i + 1])
                # Calculate mean amplitude of R-peaks up to the current index
                r_amp_mean = np.mean(r_amplitudes[:i + 1])

                # Append features and corresponding label
                all_features.append([rr_mean, rr_std, r_amp_mean])
                all_labels.append(1 if labels[i] == 'A' else 0)  # 1: Apnea, 0: Normal

        print(f"Processed record: {record_name}")

    except Exception as e:
        print(f"Error processing record {record_name}: {e}")

# Convert features and labels to numpy arrays
all_features = np.array(all_features)
all_labels = np.array(all_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_features, all_labels, test_size=0.3, random_state=42)

print(f"Training samples: {len(y_train)}, Testing samples: {len(y_test)}")

# Initialize and train a Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Loading record: a01
Loaded annotations: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',

## Check torch

In [5]:
!pip show torch

Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


# Use Pytorch

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import classification_report, accuracy_score

# Define the path to the dataset
base_path = '/root/.cache/kagglehub/datasets/bettycxh06/apnea-ecg-database/versions/7/apnea-ecg-database-1.0.0'
data_path = os.path.join(base_path, 'apnea-ecg-database-1.0.0')

# List of ECG record names to process
record_names = [
    "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
    "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
    "b01", "b02", "b03", "b04", "b05",
    "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
]

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Process each ECG record
for record_name in record_names:
    try:
        # Construct the full path to the record
        record_path = os.path.join(data_path, record_name)

        # Load the ECG signal
        record = wfdb.rdrecord(record_path)
        ecg_signal = record.p_signal[:, 0]  # Assuming ECG is in the first channel

        # Load the annotations
        annotation = wfdb.rdann(record_path, 'apn')
        labels = annotation.symbol  # 'A' for Apnea, 'N' for Normal

        # Clean the ECG signal
        ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=record.fs)

        # Detect R-peaks
        r_peaks = nk.ecg_findpeaks(ecg_cleaned, sampling_rate=record.fs)['ECG_R_Peaks']

        # Skip records with insufficient R-peaks
        if len(r_peaks) < 2:
            continue

        # Extract RR intervals (time between R-peaks) in milliseconds
        rr_intervals = np.diff(r_peaks) / record.fs * 1000

        # Extract R-peak amplitudes
        r_amplitudes = ecg_cleaned[r_peaks]

        # Aggregate features for each segment
        for i in range(min(len(labels), len(rr_intervals))):
            if labels[i] in ['A', 'N']:
                # Calculate mean and standard deviation of RR intervals up to the current index
                rr_mean = np.mean(rr_intervals[:i + 1])
                rr_std = np.std(rr_intervals[:i + 1])
                # Calculate mean amplitude of R-peaks up to the current index
                r_amp_mean = np.mean(r_amplitudes[:i + 1])

                # Append features and corresponding label
                all_features.append([rr_mean, rr_std, r_amp_mean])
                all_labels.append(1 if labels[i] == 'A' else 0)  # 1: Apnea, 0: Normal

    except Exception as e:
        print(f"Error processing record {record_name}: {e}")

# Define dataset
class ECGDataset(Dataset):
    def __init__(self, features, labels):
        # Convert features and labels to tensors in the constructor:
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create dataset instance
dataset = ECGDataset(all_features, all_labels)

# Convert features and labels to numpy arrays
all_features = np.array(all_features)
all_labels = np.array(all_labels)

# Split dataset into training (70%), validation (15%), and testing (15%)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a custom PyTorch dataset
class ECGDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create PyTorch datasets and dataloaders
train_dataset = ECGDataset(X_train, y_train)
test_dataset = ECGDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a neural network
class ECGNet(nn.Module):
    def __init__(self):
        super(ECGNet, self).__init__()
        self.fc1 = nn.Linear(3, 64)  # 3 input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # 2 output classes (Apnea, Normal)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = ECGNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop with validation
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_true = []
    val_pred = []
    with torch.no_grad():
        for features, labels in val_loader:
            outputs = model(features)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_true.extend(labels.numpy())
            val_pred.extend(predicted.numpy())
    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = accuracy_score(val_true, val_pred) * 100

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {avg_train_loss:.4f}, "
          f"Validation Loss: {avg_val_loss:.4f}, "
          f"Validation Accuracy: {val_accuracy:.2f}%")

# Testing phase
model.eval()
test_true = []
test_pred = []
with torch.no_grad():
    for features, labels in test_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        test_true.extend(labels.numpy())
        test_pred.extend(predicted.numpy())

# Print classification report and accuracy
print("Test Classification Report:")
print(classification_report(test_true, test_pred))
test_accuracy = accuracy_score(test_true, test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch [1/20], Train Loss: 2.8138, Validation Loss: 1.1369, Validation Accuracy: 61.62%
Epoch [2/20], Train Loss: 1.1617, Validation Loss: 0.5898, Validation Accuracy: 64.75%
Epoch [3/20], Train Loss: 1.5152, Validation Loss: 0.7893, Validation Accuracy: 64.36%
Epoch [4/20], Train Loss: 1.1852, Validation Loss: 0.6981, Validation Accuracy: 65.10%
Epoch [5/20], Train Loss: 0.9815, Validation Loss: 0.6570, Validation Accuracy: 65.45%
Epoch [6/20], Train Loss: 0.8873, Validation Loss: 0.5992, Validation Accuracy: 61.58%
Epoch [7/20], Train Loss: 0.7694, Validation Loss: 0.6045, Validation Accuracy: 61.62%
Epoch [8/20], Train Loss: 0.8065, Validation Loss: 0.5711, Validation Accuracy: 66.12%
Epoch [9/20], Train Loss: 0.6651, Validation Loss: 1.0045, Validation Accuracy: 65.77%
Epoch [10/20], Train Loss: 0.9088, Validation Loss: 0.5741, Validation Accuracy: 62.25%
Epoch [11/20], Train Loss: 0.6572, Validation Loss: 1.0254, Validation Accuracy: 61.62%
Epoch [12/20], Train Loss: 0.6955, Valida

# Use Pytorch with Hyperparameter tuning

In [7]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import neurokit2 as nk
import wfdb

# Define the path to the dataset
base_path = '/root/.cache/kagglehub/datasets/bettycxh06/apnea-ecg-database/versions/7/apnea-ecg-database-1.0.0'
data_path = os.path.join(base_path, 'apnea-ecg-database-1.0.0')

# List of ECG record names to process
record_names = [
    "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
    "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
    "b01", "b02", "b03", "b04", "b05",
    "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
]

# Initialize lists to store features and labels
all_features = []
all_labels = []

# Process each ECG record
for record_name in record_names:
    try:
        # Construct the full path to the record
        record_path = os.path.join(data_path, record_name)
        print(f"Loading record: {record_name}")

        # Load the ECG signal
        record = wfdb.rdrecord(record_path)
        ecg_signal = record.p_signal[:, 0]  # Assuming ECG is in the first channel

        # Load the annotations
        annotation = wfdb.rdann(record_path, 'apn')
        labels = annotation.symbol  # 'A' for Apnea, 'N' for Normal
        print(f"Loaded annotations: {labels}")

        # Clean the ECG signal
        ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=record.fs)

        # Detect R-peaks
        r_peaks = nk.ecg_findpeaks(ecg_cleaned, sampling_rate=record.fs)['ECG_R_Peaks']

        # Check if sufficient R-peaks are detected
        if len(r_peaks) < 2:
            print(f"Warning: Insufficient R-peaks detected in {record_name}. Skipping.")
            continue

        # Extract RR intervals (time between R-peaks) in milliseconds
        rr_intervals = np.diff(r_peaks) / record.fs * 1000

        # Extract R-peak amplitudes
        r_amplitudes = ecg_cleaned[r_peaks]

        # Aggregate features for each segment
        for i in range(min(len(labels), len(rr_intervals))):
            if labels[i] in ['A', 'N']:
                # Calculate mean and standard deviation of RR intervals up to the current index
                rr_mean = np.mean(rr_intervals[:i + 1])
                rr_std = np.std(rr_intervals[:i + 1])
                # Calculate mean amplitude of R-peaks up to the current index
                r_amp_mean = np.mean(r_amplitudes[:i + 1])

                # Append features and corresponding label
                all_features.append([rr_mean, rr_std, r_amp_mean])
                all_labels.append(1 if labels[i] == 'A' else 0)  # 1: Apnea, 0: Normal

        print(f"Processed record: {record_name}")

    except Exception as e:
        print(f"Error processing record {record_name}: {e}")

# Convert features and labels to numpy arrays
all_features = np.array(all_features)
all_labels = np.array(all_labels)

# Split the data into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(
    all_features, all_labels, test_size=0.4, random_state=42
)  # 60% for training, 40% for val/test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)  # Split remaining 40% equally for val and test

print(f"Training samples: {len(y_train)}, Testing samples: {len(y_test)}")

# Define a custom PyTorch dataset
class ECGDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create dataset instances
train_dataset = ECGDataset(X_train, y_train)
test_dataset = ECGDataset(X_test, y_test)

# Calculate class weights to handle class imbalance
class_counts = np.bincount(y_train)
class_weights = 1. / class_counts
sample_weights = class_weights[y_train]
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Create DataLoader with WeightedRandomSampler
val_dataset = ECGDataset(X_val, y_val)  # Create validation dataset
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False) # Create validation dataloader

test_dataset = ECGDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a neural network model with additional layers
class EnhancedNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes, dropout_rate = 0.5):
        super(EnhancedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        self.dropout = nn.Dropout(p=dropout_rate)  # Adding dropout for regularization

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout(out)  # Apply dropout after first layer
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Initialize model, loss function, and optimizer
input_size = X_train.shape[1]
hidden_size1 = 64
hidden_size2 = 32
num_classes = 2
num_epochs = 30
learning_rate = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedNN(input_size, hidden_size1, hidden_size2, num_classes).to(device)

# Use weighted cross-entropy loss to handle class imbalance
criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.1
    for features, labels in train_loader:
        # Move data to the appropriate device
        features, labels = features.to(device), labels.to(device)

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(features)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()

        # Perform a single optimization step (parameter update)
        optimizer.step()

        # Update running loss
        running_loss += loss.item()

    # Compute average loss over an epoch
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

model.eval()
test_correct = []
test_total = []
with torch.no_grad():  # Disable gradient calculation
  for test_features, test_labels in test_loader:
      test_features, test_labels = test_features.to(device), test_labels.to(device)
      test_outputs = model(test_features)
      _, test_predicted = torch.max(test_outputs, 1)

      # Extend the lists with true and predicted labels for this batch
      test_correct.extend(test_labels.cpu().numpy())
      test_total.extend(test_predicted.cpu().numpy())


# Print classification report and accuracy
print("\nTest Classification Report:")
print(classification_report(test_correct, test_total))
test_accuracy = accuracy_score(test_correct, test_total)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Loading record: a01
Loaded annotations: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A',

In [8]:
!pip install optuna

# Use Pytorch with Optuna Hyperparameter tuning

In [9]:
import optuna

# def objective(trial):
#     # Suggest hyperparameters
#     learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1, log= True)
#     hidden_size1 = trial.suggest_int('hidden_size1', 32, 128)
#     hidden_size2 = trial.suggest_int('hidden_size2', 16, 64)
#     dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
#     batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

#     # Update DataLoader with suggested batch size
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#     # Initialize the model with suggested hyperparameters
#     model = EnhancedNN(input_size, hidden_size1, hidden_size2, num_classes, dropout_rate).to(device)

#     # Define optimizer with suggested learning rate
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     # Training loop (simplified for brevity)
#     for epoch in range(num_epochs):
#         model.train()
#         for features, labels in train_loader:
#             features, labels = features.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(features)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()

def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    hidden_size1 = trial.suggest_int('hidden_size1', 32, 128)
    hidden_size2 = trial.suggest_int('hidden_size2', 16, 64)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Update DataLoader with suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model with suggested hyperparameters
    model = EnhancedNN(input_size, hidden_size1, hidden_size2, num_classes, dropout_rate).to(device)

    # Define optimizer with suggested learning rate
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop (simplified for brevity)
    for epoch in range(num_epochs):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return val_loss  # Optuna minimizes the objective

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

print('Best hyperparameters:', study.best_params)

[I 2024-12-31 19:11:54,937] A new study created in memory with name: no-name-df8c9076-4c34-4dea-ad42-378f5ba0f169
[I 2024-12-31 19:12:05,315] Trial 0 finished with value: 34.74153035879135 and parameters: {'learning_rate': 0.0006089425281205809, 'hidden_size1': 119, 'hidden_size2': 39, 'dropout_rate': 0.2880696546152522, 'batch_size': 64}. Best is trial 0 with value: 34.74153035879135.
[I 2024-12-31 19:12:15,494] Trial 1 finished with value: 29.767848432064056 and parameters: {'learning_rate': 0.0021064209979199974, 'hidden_size1': 65, 'hidden_size2': 47, 'dropout_rate': 0.3058764629784719, 'batch_size': 64}. Best is trial 1 with value: 29.767848432064056.
[I 2024-12-31 19:12:25,461] Trial 2 finished with value: 38.859352827072144 and parameters: {'learning_rate': 0.0005185218075125457, 'hidden_size1': 67, 'hidden_size2': 58, 'dropout_rate': 0.4829880097476248, 'batch_size': 64}. Best is trial 1 with value: 29.767848432064056.
[I 2024-12-31 19:12:43,046] Trial 3 finished with value: 80

Best hyperparameters: {'learning_rate': 0.0021064209979199974, 'hidden_size1': 65, 'hidden_size2': 47, 'dropout_rate': 0.3058764629784719, 'batch_size': 64}


## Use Optuna's optimized hyperparameters to train and evaluate the mode

In [14]:
# Best hyperparameters obtained from Optuna
best_params = {'learning_rate': 0.0021064209979199974,
               'hidden_size1': 65,
               'hidden_size2': 47,
               'dropout_rate': 0.3058764629784719,
               'batch_size': 64}

# Initialize the model with best hyperparameters
model = EnhancedNN(
    input_size=input_size,
    hidden_size1=best_params['hidden_size1'],
    hidden_size2=best_params['hidden_size2'],
    num_classes=num_classes,
    dropout_rate=best_params['dropout_rate']
).to(device)

# Define the optimizer with the best learning rate
optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

# Update DataLoader with the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'], shuffle=False)

# Training loop
num_epochs = 30  # You can adjust this as necessary
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.1
    for features, labels in train_loader:
        # Move data to device (GPU/CPU)
        features, labels = features.to(device), labels.to(device)

        # Forward pass: compute predictions
        outputs = model(features)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Backward pass
        loss.backward()

        # Optimization step
        optimizer.step()

        # Accumulate the running loss
        running_loss += loss.item()

    # Compute and print epoch loss
    epoch_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Evaluation phase
model.eval()  # Set the model to evaluation mode

# Validation dataset evaluation
val_true = []
val_pred = []
with torch.no_grad():
    for features, labels in val_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        val_true.extend(labels.cpu().numpy())
        val_pred.extend(predicted.cpu().numpy())

# Print validation classification report and accuracy
print("\nValidation Classification Report:")
print(classification_report(val_true, val_pred))
val_accuracy = accuracy_score(val_true, val_pred)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Test dataset evaluation
tested_true = []
tested_pred = []
with torch.no_grad():
    for features, labels in test_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        test_true.extend(labels.cpu().numpy())
        test_pred.extend(predicted.cpu().numpy())

# Print test classification report and accuracy
print("\nTest Classification Report:")
print(classification_report(test_true, test_pred))
test_accuracy = accuracy_score(test_true, test_pred)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch [1/30], Loss: 5.1785
Epoch [2/30], Loss: 2.2147
Epoch [3/30], Loss: 1.0917
Epoch [4/30], Loss: 0.8753
Epoch [5/30], Loss: 0.7200
Epoch [6/30], Loss: 0.6576
Epoch [7/30], Loss: 0.6211
Epoch [8/30], Loss: 0.5920
Epoch [9/30], Loss: 0.5436
Epoch [10/30], Loss: 0.5627
Epoch [11/30], Loss: 0.5679
Epoch [12/30], Loss: 0.5566
Epoch [13/30], Loss: 0.5474
Epoch [14/30], Loss: 0.5441
Epoch [15/30], Loss: 0.5436
Epoch [16/30], Loss: 0.5281
Epoch [17/30], Loss: 0.5422
Epoch [18/30], Loss: 0.5652
Epoch [19/30], Loss: 0.5215
Epoch [20/30], Loss: 0.5422
Epoch [21/30], Loss: 0.5175
Epoch [22/30], Loss: 0.5319
Epoch [23/30], Loss: 0.5414
Epoch [24/30], Loss: 0.5321
Epoch [25/30], Loss: 0.5247
Epoch [26/30], Loss: 0.5254
Epoch [27/30], Loss: 0.5291
Epoch [28/30], Loss: 0.5233
Epoch [29/30], Loss: 0.5194
Epoch [30/30], Loss: 0.5203

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.46      0.62      2088
           1       0.53  